In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Introduction
* Raw CSV loaded and lightly processed. Output: two column csv columns, property address, flat tag
* Data labelled in programmatic. Output: json file of entities.
* Data programmatic output json cleaned ordered and overlaps removed. Output: json file
* **Clean json converted to dataframe and multi-addresses expanded.** Output: CSV
* Count and locate addresses
* Create address matcher and match businesses
* Classify address types

This notebook is used so that I can create the code necessary to expand the addresses so that a single property/dwelling is a single line. This means the data will follow the tidy data principles of one observation per line where an observation is what is commonly thought of as a property.

In [3]:
import json
import pandas as pd
import re
from helper_functions import *
import numpy as np

In [4]:

with open('/tf/empty_homes_data/humanloop_04_05_22_t1350.json', "r") as read_file:
    all_entities_json = json.load(read_file)

# with open('/tf/empty_homes_data/full_dataset_no_overlaps.json', "r") as read_file:
#     all_entities_json = json.load(read_file)


In [6]:
#all_entities_json['datapoints']#[[int(x) for x in list(test_set['datapoint_id'])]]

In [5]:
#output_dict = [x for x in all_entities_json if int(x['data_stuff.id']) == 42353]

# Transform python object back into json
#output_json = json.dumps(output_dict)

In [7]:

all_entities = pd.json_normalize(
    all_entities_json["datapoints"],
    record_path=["programmatic", "results"],
    meta=["data", "id"],
    #record_prefix="result_stuff.",
    meta_prefix="data_stuff.",
    errors="ignore",
)
all_entities = all_entities.rename(columns = {'data_stuff.id':'datapoint_id',
                                             'text':'label_text'})

all_entities["text"] = all_entities["data_stuff.data"].map(lambda x: x["text"])

#all_entities.drop(['data_stuff.data'], axis = 1, inplace = True)

all_entities = all_entities.sort_values(['datapoint_id', 'start'])

In [8]:
# all_entities = pd.json_normalize(all_entities_json, record_path = "labels",
#                        meta = ['datapoint_id', 'text'])

all_entities['label_id_count'] = all_entities.groupby(['datapoint_id', 'label']).cumcount()

all_entities['label_text'] =all_entities['label_text'].str.replace("100-1124", "100-112")

#strips leading and trailing whitespace. this makes matching better
#but means the start and end points are incorrect
#There may be a crafty way of fixing this
all_entities['label_text'] = all_entities['label_text'].str.strip()

all_entities.drop(columns = ['data_stuff.data'], inplace = True)

In [9]:
all_entities[all_entities['datapoint_id']==88184]

,start,end,label_text,labelId,label,labellingFunctionId,groundTruthId,datapoint_id,text,label_id_count
419599,6,10,1-12,14,unit_id,53,None,88184,"flats 1-12, 176-179 shoreditch high street, lo...",0
419595,12,19,176-179,13,street_number,5,None,88184,"flats 1-12, 176-179 shoreditch high street, lo...",0
419597,20,42,shoreditch high street,12,street_name,20,None,88184,"flats 1-12, 176-179 shoreditch high street, lo...",0
419600,20,42,shoreditch high street,12,street_name,28,None,88184,"flats 1-12, 176-179 shoreditch high street, lo...",1
419596,44,50,london,5,city,10,None,88184,"flats 1-12, 176-179 shoreditch high street, lo...",0
419598,52,58,e1 6hp,11,postcode,50,None,88184,"flats 1-12, 176-179 shoreditch high street, lo...",0


## remove overlapping entities

In [ ]:

def remove_overlaps(x):
    #this functions is modified from 
    #https://stackoverflow.com/questions/57804145/combining-rows-with-overlapping-time-periods-in-a-pandas-dataframe
    x = x.copy()
    #create a unique label, this is used for joining the data back on 
    #and removes a reliance on the data being pre-sorted
    x['unique_label'] = [*range(0,x.shape[0])]
    #get the size of the spans
    x['diff'] = (x['end']-x['start'])

    
    startdf = pd.DataFrame({'position':x['start'], 'unique_label':x['unique_label'], 'what':1})
    enddf = pd.DataFrame({'position':x['end'], 'unique_label':x['unique_label'], 'what':-1})
    mergdf = pd.concat([startdf, enddf]).sort_values('position')
    mergdf['running'] = mergdf['what'].cumsum()
    mergdf['newwin'] = mergdf['running'].eq(1) & mergdf['what'].eq(1)
    mergdf['group'] = mergdf['newwin'].cumsum()
    
    #merge back on using uniqe label to ensure correct ordering
    x = x.merge(mergdf.loc[mergdf['what'].eq(1),['unique_label','group']], how = 'left', on = 'unique_label')
    #sort within group and keep only the largest
    x = x.sort_values('diff', ascending=False).groupby(['group', 'datapoint_id'], as_index=False).first()

    x.drop(['diff', 'unique_label', 'group'], axis = 1, inplace = True)
    
    x.reset_index(drop = True, inplace = True)

    return(x)


#ddf = remove_overlaps(df)
#takes about 20 minutes
all_entities = all_entities.groupby(['datapoint_id']).apply(remove_overlaps)


In [11]:
# def remove_overlapping_spans(
#     spans: pd.DataFrame, groupby="datapoint_id", start="start", end="end"
# ):
#     """Removes rows from a DataFrame where start:end overlap.

#     Attempts to keep the longer of the overlapping spans.
#     """
#     spans_to_remove = []
#     for datapoint_id, datapoint_spans in spans.groupby(groupby):
#         intervals = datapoint_spans.apply(
#             lambda x: pd.Interval(left=getattr(x, start), right=getattr(x, end)), axis=1
#         )
#         for i, (index_a, interval_a) in enumerate(intervals.iteritems()):
#             for j, (index_b, interval_b) in enumerate(
#                 intervals.iloc[i + 1 :].iteritems()
#             ):
#                 if interval_a.overlaps(interval_b):
#                     # print(i, j, index_a, index_b, interval_a, interval_b)
#                     # Overlapping ground truths at index_a and index_b.
#                     # Keep only longer of the two.
#                     if interval_a.length >= interval_b.length:
#                         spans_to_remove.append(index_b)
#                     else:
#                         spans_to_remove.append(index_a)

#     return spans[~spans.index.isin(spans_to_remove)]

# import json


# #1345
# with open('/tf/empty_homes_data/humanloop_04_04_22_t1210.json', "r") as f:
#     data = json.load(f)

# df = pd.json_normalize(
#     data["datapoints"],
#     # record_path=["programmatic", "aggregateResults"],
#     record_path=["programmatic", "results"],
#     meta=["data", "id"],
#     record_prefix="result_stuff.",
#     meta_prefix="data_stuff.",
#     errors="ignore",
# )
# df["full_text"] = df["data_stuff.data"].map(lambda x: x["text"])

# df = all_entities.copy()



# new_df = remove_overlapping_spans(
#     df, groupby="data_stuff.id", start="result_stuff.start", end="result_stuff.end"
# )

In [21]:


# test_set = pd.read_csv('/tf/empty_homes_data/test_set_indices.csv')

# all_entities[all_entities['datapoint_id'].isin(test_set['datapoint_id'])].to_csv('/tf/empty_homes_data/ground_truth.csv')

## Example of the data frame of labels

In [12]:
all_entities

datapoint_id  start  end                      label_text  \
datapoint_id                                                               
0            0             0      0   25       westleigh lodge care home   
             1             0     27   39                    nel pan lane   
             2             0     41   47                           leigh   
             3             0     48   55                         wn7 5jt   
1            0             1      0    4                            flat   
...                      ...    ...  ...                             ...   
94087        1         94087     12   42  discovery dock apartments east   
             2         94087     44   45                               3   
             3         94087     46   63               south quay square   
             4         94087     65   72                          london   
             5         94087     73   80                         e14 9ru   

               labelId          label  labellingFunctionId groundTruthId  \
datapoint_id                                                               
0            0      10  building_name                   30          None   
             1      12    street_name                   24          None   
             2       3           city                   12          None   
             3      11       postcode                   36          None   
1            0      15      unit_type                   18          None   
...                ...            ...                  ...           ...   
94087        1      10  building_name                   54          None   
             2      13  street_number                    9          None   
             3      12    street_name                   22          None   
             4       3           city                   12          None   
             5      11       postcode                   36          None   

                                                             text  \
datapoint_id                                                        
0            0  westleigh lodge care home, nel pan lane, leigh...   
             1  westleigh lodge care home, nel pan lane, leigh...   
             2  westleigh lodge care home, nel pan lane, leigh...   
             3  westleigh lodge care home, nel pan lane, leigh...   
1            0       flat 1, 1a canal street, manchester (m1 3he)   
...                                                           ...   
94087        1  storage 17, discovery dock apartments east, 3 ...   
             2  storage 17, discovery dock apartments east, 3 ...   
             3  storage 17, discovery dock apartments east, 3 ...   
             4  storage 17, discovery dock apartments east, 3 ...   
             5  storage 17, discovery dock apartments east, 3 ...   

                label_id_count  
datapoint_id                    
0            0               0  
             1               0  
             2               0  
             3               0  
1            0               0  
...                        ...  
94087        1               0  
             2               0  
             3               0  
             4               0  
             5               0  

[414327 rows x 10 columns]

## Identify multi versus single address observations

Some addresses may have the form xx to yy but should not be expanded as this is a building that covers multiple street numbers. Items such as these need to be carefully removed before expansion

In [18]:
xx_to_yy_regex = r'^\d+(\s)?(-|to)(\s)?\d+$'

multi_check_df = all_entities[['datapoint_id', 'text', ]].drop_duplicates()
multi_check_df['comma_count'] = multi_check_df['text'].str.count(',')
multi_check_df['land'] = multi_check_df['text'].str.contains(r"^(land|plot|airspace|car|parking)", case = False)

multi_check_df['business'] = multi_check_df['text'].str.contains(r"(cinema)|(hotel)|(office)|centre|(\bpub)|holiday(\s)?inn|travel(\s)?lodge|(business)|(cafe)|(^shop)|( shop)|(restaurant)|(home)|(^store(s)?\b)|(^storage\b)|(company)|(ltd)|(limited)|(plc)|(retail)|(leisure)|(industrial)|(hall of)|trading|commercial|works", case = False)
temp_df = all_entities[['datapoint_id', 'label']].groupby(['datapoint_id', 'label']).value_counts().to_frame(name = "counts").reset_index().pivot(index = 'datapoint_id', columns = 'label', values = 'counts').fillna(0)
#test['datapoint_id'] = test.index

xx_to_yy_street_counts = all_entities['datapoint_id'][all_entities['label_text'].str.contains(
    xx_to_yy_regex)& (all_entities['label']=="street_number")
                            ].to_frame(name = 'datapoint_id').groupby('datapoint_id').size().to_frame(name = 'xx_to_yy_street_counts')

xx_to_yy_unit_counts = all_entities['datapoint_id'][all_entities['label_text'].str.contains(
    xx_to_yy_regex)& (all_entities['label']=="unit_id")
                            ].to_frame(name = 'datapoint_id').groupby('datapoint_id').size().to_frame(name = 'xx_to_yy_unit_counts')

multi_check_df = multi_check_df.merge(temp_df, how = 'left', left_on = "datapoint_id", right_index = True).\
merge(xx_to_yy_street_counts, how = 'left', left_on = "datapoint_id", right_index = True).\
merge(xx_to_yy_unit_counts, how = 'left', left_on = "datapoint_id", right_index = True).fillna(0)


del xx_to_yy_street_counts
del xx_to_yy_unit_counts

#separate the classes using logical rules
multi_check_df['class'] = np.select(
    [
        multi_check_df['land']== True,
        multi_check_df['business']== True,
        (multi_check_df['building_name']==1) & (multi_check_df['unit_id'] == 0), #this has to go infront of 'multi_check_df['xx_to_yy_unit_counts']>0'
        multi_check_df['xx_to_yy_unit_counts']>0,
        multi_check_df['street_number']>1,
        multi_check_df['unit_id']>1,
        (multi_check_df['street_number']<=1) & (multi_check_df['xx_to_yy_street_counts']<=1) & (multi_check_df['unit_id']<=1) ##This does most of the heavy lifting
    ], 
    [
        'single',
        'single',
        'single',
        'multi',
        'multi',
        'multi',
        'single',
        
    ], 
    default='unknown'
)
multi_check_df


<ipython-input-18-8da8733f2f27>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  multi_check_df['land'] = multi_check_df['text'].str.contains(r"^(land|plot|airspace|car|parking)", case = False)
<ipython-input-18-8da8733f2f27>:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  multi_check_df['business'] = multi_check_df['text'].str.contains(r"(cinema)|(hotel)|(office)|centre|(\bpub)|holiday(\s)?inn|travel(\s)?lodge|(business)|(cafe)|(^shop)|( shop)|(restaurant)|(home)|(^store(s)?\b)|(^storage\b)|(company)|(ltd)|(limited)|(plc)|(retail)|(leisure)|(industrial)|(hall of)|trading|commercial|works", case = False)
<ipython-input-18-8da8733f2f27>:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  xx_to_yy_street_counts = all_entities[

,datapoint_id,text,comma_count,land,business,building_name,city,number_filter,postcode,street_name,street_number,unit_id,unit_type,xx_to_yy_street_counts,xx_to_yy_unit_counts,class
0,0,"westleigh lodge care home, nel pan lane, leigh...",2,False,True,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,single
4,1,"flat 1, 1a canal street, manchester (m1 3he)",2,False,False,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,single
9,2,"flat 201, 1 regent road, manchester (m3 4ay)",2,False,False,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,single
14,3,"land at 2a gerard street, ashton in makerfield...",2,True,False,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,single
18,4,"unit 111, timber wharf, worsley street, manche...",3,False,False,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414305,94083,"2nd floor, 52 lime street, london (ec3m 7aw)",2,False,False,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,single
414309,94084,"155 bishopsgate, london (ec2m 3ad)",1,False,False,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,single
414313,94085,"9th and 10th floors, 52 lime street, london (e...",2,False,False,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,single
414317,94086,"part of tenth floor, 6 bevis marks, london (ec...",2,False,False,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,single


### Get the indexes of multi properties

In [19]:
multi_unit_id = set(multi_check_df['datapoint_id'][(multi_check_df['class']=='multi') &( multi_check_df['unit_id']>0)].tolist())
multi_property = set(multi_check_df['datapoint_id'][(multi_check_df['class']=='multi') &( multi_check_df['unit_id']==0)].tolist())
all_multi_ids = list(multi_unit_id) +list(multi_property)
multi_check_df.groupby('class').size()

class
multi      5194
single    88894
dtype: int64

# checking weird regex problems

In [ ]:
all_entities['label_text'].to_list()[0]

## Spread label data

In [20]:
#pivot the columns so that each label class is it's own column and the value in the column is the text

temp_df = all_entities[all_entities.datapoint_id.isin(all_multi_ids)]

temp_df['index'] = temp_df.index
df = temp_df[['index', 'label', 'label_text']].pivot(index='index',columns='label',values='label_text')
#add the datapoint_id back in for each of joining
df = pd.concat([temp_df['datapoint_id'], df], axis=1).merge(temp_df[['datapoint_id' ,'text']].drop_duplicates(), 
          how = "left",
          left_on = "datapoint_id", right_on = "datapoint_id")
del temp_df

<ipython-input-20-8bdee3a0bd7d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['index'] = temp_df.index


In [ ]:
#df[df.text.str.contains('unit')==True].drop_duplicates()#.to_csv('/tf/empty_homes_data/delete_me.csv')

## Add blockers

Blockers prevent the filling of wrong information. As an example if a building is going to back fill up previous addresses it should not back fill past another street as this is highly unlikely to be the same building

In [21]:
df['building_name'][df['street_name'].notnull()] = 'block'
df['street_number'][df['street_name'].notnull()] = 'block' #for multi-flats inside a common building

#returns true if current number filter is null and the next row has street_number or unit id is not null
#prevents number filters propergsating back across roads and unit ids
number_filter_block = df['number_filter'].isnull() & (df['street_number'].shift().notnull() |df['unit_id'].shift().notnull())
df['number_filter'][number_filter_block] = 'block'

<ipython-input-21-b3dd40429e14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['building_name'][df['street_name'].notnull()] = 'block'
<ipython-input-21-b3dd40429e14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['street_number'][df['street_name'].notnull()] = 'block' #for multi-flats inside a common building
<ipython-input-21-b3dd40429e14>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['number_filter'][number_filter_bl

## Backfill 

Backfilling adds address information in. However, street address should only be back filled for multi addresses.
I need to work out how to do flat, which may be before or after the unit ID
Also I don't think this is a very good way of doing it at all. Using the pre-spread list is probably better and only working on the multi-addresses is probably a much faster and cleaner way. But I will have to think about how to do it

In [22]:
df['number_filter'] = df[['datapoint_id','number_filter']].groupby('datapoint_id').fillna(method ='bfill')
df['building_name'] = df[['datapoint_id','building_name']].groupby('datapoint_id').fillna(method ='bfill')
df['street_number'] = df[['datapoint_id','street_number']].groupby('datapoint_id').fillna(method ='bfill')
df['postcode'] = df[['datapoint_id','postcode']].groupby('datapoint_id').fillna(method ='bfill')
df['street_name'] = df[['datapoint_id','street_name']].groupby('datapoint_id').fillna(method ='bfill')
df['number_filter'] = df[['datapoint_id','number_filter']].groupby('datapoint_id').fillna(method ='bfill')
df['city'] = df[['datapoint_id','city']].groupby('datapoint_id').fillna(method ='bfill')
df['unit_type'] = df[['datapoint_id','unit_type']].groupby('datapoint_id').fillna(method ='ffill') #should this will backwards or forwards? as mostly it is flat xx not xx flat?

In [ ]:
df

In [23]:
expanded_street = df[df.datapoint_id.isin(multi_property) & df.street_number.str.contains(xx_to_yy_regex)].reset_index()
expanded_unit_id = df[df.datapoint_id.isin(multi_unit_id) & df.unit_id.str.contains(xx_to_yy_regex)].reset_index()

expanded_street = expand_dataframe_numbers(expanded_street, column_name = "street_number" )
expanded_unit_id = expand_dataframe_numbers(expanded_unit_id, column_name = "unit_id" )

<ipython-input-23-6dd2bb121964>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  expanded_street = df[df.datapoint_id.isin(multi_property) & df.street_number.str.contains(xx_to_yy_regex)].reset_index()
<ipython-input-23-6dd2bb121964>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  expanded_unit_id = df[df.datapoint_id.isin(multi_unit_id) & df.unit_id.str.contains(xx_to_yy_regex)].reset_index()


i= 0  expand time,0.001 filter time0.0 make_dataframe_time 0.001
i= 1000  expand time,0.203 filter time0.127 make_dataframe_time 0.985
i= 2000  expand time,0.387 filter time0.244 make_dataframe_time 1.873
i= 0  expand time,0.001 filter time0.0 make_dataframe_time 0.001
i= 1000  expand time,0.19 filter time0.122 make_dataframe_time 1.025


In [24]:
#unit id and street number that does does not have the xx to yy format and so has already been expanded by spreaing and backfilling
expanded_street_simple = df[df.datapoint_id.isin(multi_property) & (df.street_number.str.contains(xx_to_yy_regex)==False) & (df.street_number!='block')].reset_index()
expanded_unit_id_simple = df[df.datapoint_id.isin(multi_unit_id) & (df.unit_id.str.contains(xx_to_yy_regex)==False) & (df.unit_id!='block')].reset_index()

<ipython-input-24-53d39f72e523>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  expanded_street_simple = df[df.datapoint_id.isin(multi_property) & (df.street_number.str.contains(xx_to_yy_regex)==False) & (df.street_number!='block')].reset_index()
<ipython-input-24-53d39f72e523>:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  expanded_unit_id_simple = df[df.datapoint_id.isin(multi_unit_id) & (df.unit_id.str.contains(xx_to_yy_regex)==False) & (df.unit_id!='block')].reset_index()


In [25]:
#pivot the columns so that each label class is it's own column and the value in the column is the text

#remove the multi-addresses
single_address_only =all_entities[~all_entities['datapoint_id'].isin(all_multi_ids)]
#remove all but the first instance of a label in the remaining instances
#this is because for single addresses there should be only a single label for each class
single_address_only =single_address_only[single_address_only['label_id_count']==0]
df2 = single_address_only.pivot(index='datapoint_id',columns='label',values='label_text')
#add the datapoint_id back in for each of joining
df2 = df2.merge(single_address_only[['datapoint_id', 'text']].drop_duplicates(), 
          how = "left",
          left_on = "datapoint_id", right_on = "datapoint_id")

df2

,datapoint_id,building_name,city,number_filter,postcode,street_name,street_number,unit_id,unit_type,text
0,0,westleigh lodge care home,leigh,NaN,wn7 5jt,nel pan lane,NaN,NaN,NaN,"westleigh lodge care home, nel pan lane, leigh..."
1,1,NaN,manchester,NaN,m1 3he,canal street,1a,NaN,flat,"flat 1, 1a canal street, manchester (m1 3he)"
2,2,NaN,manchester,NaN,m3 4ay,regent road,1,NaN,flat,"flat 201, 1 regent road, manchester (m3 4ay)"
3,3,NaN,wigan,NaN,wn4 9aa,gerard street,NaN,NaN,land,"land at 2a gerard street, ashton in makerfield..."
4,4,NaN,manchester,NaN,m15 4nz,worsley street,NaN,111,unit,"unit 111, timber wharf, worsley street, manche..."
...,...,...,...,...,...,...,...,...,...,...
88889,94083,NaN,london,NaN,ec3m 7aw,lime street,52,NaN,NaN,"2nd floor, 52 lime street, london (ec3m 7aw)"
88890,94084,NaN,london,NaN,ec2m 3ad,bishopsgate,155,NaN,NaN,"155 bishopsgate, london (ec2m 3ad)"
88891,94085,NaN,london,NaN,ec3m 7af,lime street,52,NaN,NaN,"9th and 10th floors, 52 lime street, london (e..."
88892,94086,NaN,london,NaN,ec3a 7ba,bevis marks,6,NaN,NaN,"part of tenth floor, 6 bevis marks, london (ec..."


In [26]:
full_expanded_data = pd.concat([expanded_street, 
           expanded_unit_id, 
           expanded_street_simple, 
           expanded_unit_id_simple, 
           df2, ])

In [27]:
ocod_data =  pd.read_csv('/tf/empty_homes_data/' +
                    'OCOD_FULL_2022_02.csv',
                   encoding_errors= 'ignore').rename(columns = lambda x: x.lower().replace(" ", "_"))
#empty addresses cannot be used. however there are only three so not a problem
ocod_data = ocod_data.dropna(subset = 'property_address')
ocod_data.reset_index(inplace = True, drop = True)
ocod_data = ocod_data[['title_number', 'tenure', 'district', 'county',
       'region', 'multiple_address_indicator', 'price_paid', 'property_address']]

<ipython-input-27-83c4912c1574>:1: DtypeWarning: Columns (24,28,30,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  ocod_data =  pd.read_csv('/tf/empty_homes_data/' +


In [28]:
full_expanded_data = full_expanded_data.merge(ocod_data, how = "left", left_on = "datapoint_id", right_index = True)
full_expanded_data['property_address'].str.lower().equals(full_expanded_data['text']) # When 'True' this shows the match works because the address string are equal

False

In [29]:
#Why is this 1 address not the same?

full_expanded_data.loc[~((full_expanded_data['text']==full_expanded_data['property_address'].str.lower())), ['property_address', 'text', 'datapoint_id']]

,property_address,text,datapoint_id
0,"202, Stanley Court 19-23, Stanley Street, Live...","202, stanley court, 19-23, stanley street, liv...",3102
1,"202, Stanley Court 19-23, Stanley Street, Live...","202, stanley court, 19-23, stanley street, liv...",3102
2,"202, Stanley Court 19-23, Stanley Street, Live...","202, stanley court, 19-23, stanley street, liv...",3102
3,"202, Stanley Court 19-23, Stanley Street, Live...","202, stanley court, 19-23, stanley street, liv...",3102
4,"202, Stanley Court 19-23, Stanley Street, Live...","202, stanley court, 19-23, stanley street, liv...",3102
8589,"40a, 40, 40¨, 42, 44 East Bond Street, 2, 4 Gr...","40a, 40, 40 ̈, 42, 44 east bond street, 2, 4 g...",57998
8590,"40a, 40, 40¨, 42, 44 East Bond Street, 2, 4 Gr...","40a, 40, 40 ̈, 42, 44 east bond street, 2, 4 g...",57998
8591,"40a, 40, 40¨, 42, 44 East Bond Street, 2, 4 Gr...","40a, 40, 40 ̈, 42, 44 east bond street, 2, 4 g...",57998
8592,"40a, 40, 40¨, 42, 44 East Bond Street, 2, 4 Gr...","40a, 40, 40 ̈, 42, 44 east bond street, 2, 4 g...",57998
8593,"40a, 40, 40¨, 42, 44 East Bond Street, 2, 4 Gr...","40a, 40, 40 ̈, 42, 44 east bond street, 2, 4 g...",57998


In [30]:
full_expanded_data.loc[:, 'property_address'].str.lower().loc[full_expanded_data['datapoint_id']==57998]

8589    40a, 40, 40¨, 42, 44 east bond street, 2, 4 gr...
8590    40a, 40, 40¨, 42, 44 east bond street, 2, 4 gr...
8591    40a, 40, 40¨, 42, 44 east bond street, 2, 4 gr...
8592    40a, 40, 40¨, 42, 44 east bond street, 2, 4 gr...
8593    40a, 40, 40¨, 42, 44 east bond street, 2, 4 gr...
8594    40a, 40, 40¨, 42, 44 east bond street, 2, 4 gr...
8595    40a, 40, 40¨, 42, 44 east bond street, 2, 4 gr...
Name: property_address, dtype: object

In [31]:
full_expanded_data['within_title_id'] = full_expanded_data.groupby('title_number').cumcount()+1
full_expanded_data['unique_id'] = [str(x) + '-' + str(y) for x, y in zip(full_expanded_data['title_number'], full_expanded_data['within_title_id'])]

tmp_df =((full_expanded_data[['title_number', 'within_title_id']].groupby('title_number').max('within_title_id'))>1)
tmp_df.columns = tmp_df.columns.str.replace('within_title_id', 'within_larger_title') #could also be called nested_address
full_expanded_data = full_expanded_data.merge(tmp_df, how = "left", left_on = "title_number", right_index = True)


full_expanded_data['postcode'] =full_expanded_data['postcode'].str.upper()
del tmp_df

#re-order the columns and drop columns that are not needed

full_expanded_data =full_expanded_data[['title_number', 'within_title_id', 'unique_id', 'within_larger_title',  'tenure','unit_id', 'unit_type','building_name','street_number', 'street_name', 'postcode','city',  'district', 'county', 'region',
       'multiple_address_indicator', 'price_paid' ,'property_address']].replace('block', np.NaN)

#save as CSV
full_expanded_data.to_csv("/tf/empty_homes_data/OCOD_cleaned_expanded2.csv")

In [33]:
full_expanded_data#[full_expanded_data['street_name'].isnull()]['property_address'].to_csv('/tf/empty_homes_data/street_is_null.csv')

,title_number,within_title_id,unique_id,within_larger_title,tenure,unit_id,unit_type,building_name,street_number,street_name,postcode,city,district,county,region,multiple_address_indicator,price_paid,property_address
0,CB400630,1,CB400630-1,True,Freehold,NaN,NaN,NaN,2,miller way,NaN,peterborough,CITY OF PETERBOROUGH,CITY OF PETERBOROUGH,EAST ANGLIA,N,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (..."
1,CB400630,2,CB400630-2,True,Freehold,NaN,NaN,NaN,4,miller way,NaN,peterborough,CITY OF PETERBOROUGH,CITY OF PETERBOROUGH,EAST ANGLIA,N,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (..."
2,CB400630,3,CB400630-3,True,Freehold,NaN,NaN,NaN,6,miller way,NaN,peterborough,CITY OF PETERBOROUGH,CITY OF PETERBOROUGH,EAST ANGLIA,N,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (..."
3,CB400630,4,CB400630-4,True,Freehold,NaN,NaN,NaN,8,miller way,NaN,peterborough,CITY OF PETERBOROUGH,CITY OF PETERBOROUGH,EAST ANGLIA,N,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (..."
4,CB400630,5,CB400630-5,True,Freehold,NaN,NaN,NaN,10,miller way,NaN,peterborough,CITY OF PETERBOROUGH,CITY OF PETERBOROUGH,EAST ANGLIA,N,NaN,"2-24 (even) Miller Way, 15-25 Hammonds Drive (..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88889,AGL470128,1,AGL470128-1,False,Leasehold,NaN,NaN,NaN,52,lime street,EC3M 7AW,london,CITY OF LONDON,GREATER LONDON,GREATER LONDON,N,NaN,"2nd Floor, 52 Lime Street, London (EC3M 7AW)"
88890,AGL473126,1,AGL473126-1,False,Leasehold,NaN,NaN,NaN,155,bishopsgate,EC2M 3AD,london,CITY OF LONDON,GREATER LONDON,GREATER LONDON,N,NaN,"155 Bishopsgate, London (EC2M 3AD)"
88891,AGL473595,1,AGL473595-1,False,Leasehold,NaN,NaN,NaN,52,lime street,EC3M 7AF,london,CITY OF LONDON,GREATER LONDON,GREATER LONDON,N,NaN,"9th and 10th floors, 52 Lime Street, London (E..."
88892,AGL475468,1,AGL475468-1,False,Leasehold,NaN,NaN,NaN,6,bevis marks,EC3A 7BA,london,CITY OF LONDON,GREATER LONDON,GREATER LONDON,N,NaN,"part of Tenth Floor, 6 Bevis Marks, London (EC..."


In [ ]:
full_expanded_data[full_expanded_data['property_address'].str.contains('Scottish Provident')]

In [ ]:
300000*700